In [ ]:
import matplotlib.pyplot as plt
import cv2
import numpy as np
import random

In [ ]:
img = cv2.imread("../images/home.jpg")

def img_to_grid(img, row, col):
    rowList = [[i.min(), i.max()] for i in np.array_split(range(img.shape[0]), row)]
    colList = [[i.min(), i.max()] for i in np.array_split(range(img.shape[1]), col)]
    print(colList)
    grid = [img[j:jj,i:ii,:] for j,jj in rowList for i,ii in colList]
    return grid

def plot_grid(grid, row, col, h=4, w=5):
    fig, ax = plt.subplots(nrows=row, ncols=col)
    [axi.set_axis_off() for axi in ax.ravel()]

    fig.set_figheight(h)
    fig.set_figwidth(w)
    c = 0
    for row in ax:
        for col in row:
            col.imshow(np.flip(grid[c], axis=-1))
            c+=1
    plt.show()

if __name__=='__main__':
    row, col = 7,5
    grid = img_to_grid(img, row, col)
    plot_grid(grid, row, col)

In [ ]:
img = cv2.imread('../images/home.jpg')

width = img.shape[1]
height = img.shape[0]

top = random.choice(['male', 'female', 'flat'])
right = random.choice(['male', 'female', 'flat'])
bottom = random.choice(['male', 'female', 'flat'])
left = random.choice(['male', 'female', 'flat'])

offset = 35
points = []

# Add the top left corner point
points.append((offset, offset))

# Add the top side points
if top != 'flat':
    points.append((width//2 - offset, offset))
    if top == 'male':
        points.append((width // 2 - offset, 0))
        points.append((width // 2 + offset, 0))
    else:
        points.append((width // 2 - offset, offset * 2))
        points.append((width // 2 + offset, offset * 2))
    points.append((width//2 + offset, offset))

# Add the top right corner point
points.append((width - offset, offset))

# Add the right side points
if right != 'flat':
    points.append((width - offset, height//2 - offset))
    if top == 'male':
        points.append((width, height//2 - offset))
        points.append((width, height//2 + offset))
    else:
        points.append((width - offset*2, height//2 - offset))
        points.append((width - offset*2, height//2 + offset))
    points.append((width - offset, height//2 + offset))


# Add the bottom right corner point
points.append((width - offset, height - offset))

# Add the bottom side points
if bottom != 'flat':
    points.append((width//2 + offset, height - offset))
    if top == 'male':
        points.append((width//2 + offset, height))
        points.append((width//2 - offset, height))
    else:
        points.append((width//2 + offset, height - offset*2))
        points.append((width//2 - offset, height - offset*2))
    points.append((width//2 - offset, height - offset))

# Add the bottom left corner point
points.append((offset, height - offset))

# Add the left side points
if left != 'flat':
    points.append((offset, height//2 + offset))
    if top == 'male':
        points.append((offset*2, height//2 + offset))
        points.append((offset*2, height//2 - offset))
    elif top == 'female':
        points.append((0, height//2 + offset))
        points.append((0, height//2 - offset))
    points.append((offset, height//2 - offset))

# Convert the points to a numpy array
points = np.array(points, np.int32)
# Create a mask for the piece shape
    
mask = np.zeros((height, width, 3), np.uint8)
cv2.fillPoly(mask, [points], (255, 255, 255))

img = cv2.bitwise_and(img, mask)
cv2.imshow('Puzzle Piece', img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
image_path = "../images/puzzle piece1.png"

In [ ]:
image = cv2.imread(image_path)
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
_, thresholded = cv2.threshold(gray, 220, 255, cv2.THRESH_BINARY)

mask = cv2.bitwise_not(thresholded)

black_background = np.zeros_like(image)
black_background[mask > 0] = 255

kernel = np.ones((5,5),np.uint8)
mask = cv2.morphologyEx(black_background, cv2.MORPH_CLOSE, kernel)

cv2.imshow('Object in White, Background in Black', mask)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [ ]:
image = cv2.imread(image_path)
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
_, thresholded = cv2.threshold(gray, 220, 255, cv2.THRESH_BINARY)

mask = cv2.bitwise_not(thresholded)

black_background = np.zeros_like(image)
black_background[mask > 0] = 255

kernel = np.ones((5, 5), np.uint8)
mask_closed = cv2.morphologyEx(black_background, cv2.MORPH_CLOSE, kernel)

cv2.imshow('Object in White, Background in Black', mask_closed)

# Skeletonization
element = cv2.getStructuringElement(cv2.MORPH_CROSS, (3, 3))
done = False
size = np.size(image)
skel = np.zeros_like(image, dtype=np.uint8)

while not done:
    eroded = cv2.erode(mask_closed, element)
    temp = cv2.dilate(eroded, element)
    temp = cv2.subtract(mask_closed, temp)
    skel = cv2.bitwise_or(skel, temp)
    mask_closed = eroded.copy()
    zeros = size - cv2.countNonZero(cv2.cvtColor(mask_closed, cv2.COLOR_BGR2GRAY))
    if zeros == size:
        done = True

cv2.imshow('Skeleton', skel)
cv2.waitKey(0)
cv2.destroyAllWindows()